# Forward mode automatic differentiation

---
<div style="text-align: justify;">

Automatic differentiation (AD) is a set of techniques to compute derivatives of functions in a computer program efficiently and accurately. Given a differentiable function $f: \mathbb{R}^n \to \mathbb{R}^m$, AD seeks to compute the $m\times n$ Jacobian matrix $J_{ij} = \partial_{j}f_i$. Our convention here is that each column $j$ is the gradient vector $\partial_{j} f$. If the domain is small and $f$ itself is computationally cheap, it may be efficient to compute each column separately. Denoting the computational cost of $f$ by $[f]$, this results in $\mathcal{O}(n\times[f])$ operations. Methods of this sort fall under the bracket of forward-mode AD (FWAD). When $n$ becomes large compared to $m$, it may be more efficient to consider other avenues. A popular method is backpropagation, where each row of the Jacobian (this is all $n$ derivatives corresponding to a single component of the function) is computed separately in a single pass. Methods such as these are called reverse-mode AD (RSAD), and carry a complexity of $\mathcal{O}(m\times[f])$.

We focus here on FWAD, and in particular, its implementation through dual numbers. A dual number $d$ is an ordered pair of real numbers $(a, b)$, with $a$ the real component and $b$ the dual component. Its arithmetic is defined as
\begin{align*}
&(a, b) + (c, d) = (a + c, b + d) \\
&(a, b) \times (c, d) = (ac, ad + bc)
\end{align*}
Note the similarity with complex numbers $z$, which can also represented by an ordered pair of reals, $z = (a, b)$, and have an arithmetic of 
\begin{align*}
&(a, b) + (c, d) = (a + c, b + d) \\
&(a, b) \times (c, d) = (ac - bd, ad + bc)
\end{align*}
The only difference is in the first term in the multiplication rule. By introducing the imaginary number $i$ that satisfies $i^2= - 1$, complex numbers can be represented as $z = a + bi$. A similar innovation can be applied to dual numbers, with $i$ replaced by the Grassmann number $\epsilon$. $\epsilon$ is nilpotent, with $\epsilon^2 = 0$. Showing that this is consistent with the arithmetic rules is left to the reader as an exercise. We will denote by $\mathbb{D}$ the space of dual numbers.

Dual numbers allow us to compute exact derivatives of functions. For simplicity, we focus this discussion on $n, m = 1$. We will consider the general case at the end. Let $f:\mathbb{R}\to\mathbb{R}$ be a smooth, real-valued function over the reals. We can extend it to be a dual-valued function over dual numbers by its Taylor series,
\begin{equation*}
f(a + b\epsilon) = \sum_{i=0}^{n} \frac{f^{(n)}(a)}{n!}(b\epsilon)^n \,,
\end{equation*}
where now $f: \mathbb{D}\to\mathbb{D}$. Since $\epsilon^2 = 0$, this truncates to
\begin{equation*}
f(a + b\epsilon) = f(a) + f^{(1)}(a)b\epsilon \,.
\end{equation*}
Setting $b = 1$, we find the dual part to be the derivative $f^{(1)}(a)$. By computing the extensions of functions to the duals, we are able to extract their (exact!) derivatives automatically. 

To generalise, define the $n$-dimensional dual space as $\mathbb{D}^n = \{a + b\epsilon: a, b \in \mathbb{R}^n\}$. We extend the smooth function $f: \mathbb{R}^n \to \mathbb{R}^m$ to the duals through its Taylor expansion as before, and find
\begin{equation*}
f(a + b\epsilon) = f(a) + \epsilon J(a)\cdot b \,,
\end{equation*}
where $J(a)$ is the Jacobian evaluated at $a$. The dual part of this expression is the derivative of $f$ at $a$ along the direction of $b$. Running this calculation $n$ times, each taking $b$ to be a unit vector on $\mathbb{R}^n$, then provides the full Jacobian matrix.

It remains to implement dual numbers computationally. We will do this for the $n,m=1$ case for clarity. The steps are as follows:
1) Define a dual number class that provides overrides of the standard arithmetic operators.
2) Provide a wrapper for functions defined over real numbers to dual numbers.
3) Implement the derivative operator as the dual component of a function $f$ evaluated at $d = a + \epsilon$.

For 1), we will present here the remaining arithmetic properties of dual numbers: division and exponentiation. Firstly, division. Let $a,b,c,d$ be generic real numbers, then
\begin{equation*}
\frac{a+b\epsilon}{c+d\epsilon} = \frac{(a+b\epsilon)(c-d\epsilon)}{c^2} = \frac{ac - (ad - bc)\epsilon}{c^2} \,.
\end{equation*}
Note that this is only valid for $c \neq 0$. If $c = 0$, the equation $a+b\epsilon = ud\epsilon$ is solvable for $u$ with $u=b/d$ provided $a=0$ and $d\neq 0$. Technically, $b/d + v\epsilon$ for any real number $v$ is a solution in this case: division by a dual number with 0 real part is not well-defined. In our implementation, we will restrict $v=0$. 

For exponentiation, let $c$ be a real number. We have, by Taylor expansion,
\begin{equation*}
(a + b\epsilon)^c = a^c + c b a^{c-1}\epsilon \,.
\end{equation*}
We can extend this to exponentiation by dual numbers using the trick
\begin{equation*}
(a + b\epsilon)^{c + d\epsilon} = \exp((c + d\epsilon)\log (a+b\epsilon)) \,.
\end{equation*}
By Taylor, 
\begin{equation*}
\log(a + b\epsilon) = \log(a) + \frac{b}{a}\epsilon \,,
\end{equation*}
which requires $a > 0$. Appealing to Taylor again, we have
\begin{equation*}
(a + b\epsilon)^{c+d\epsilon} = \exp\left(c\log(a) +\left(\frac{bc}{a} + d\log(a)\right)\epsilon\right) = a^c\left(1 +\left(\frac{bc}{a} + d\log(a)\right)\epsilon \right) \,.
\end{equation*}
We may extend this for $a < 0$ and consider complexified dual numbers, but this will not be done here for brevity.

</div>

## Implementation

In [1]:
import numpy as np
from typing import Callable

from __future__ import annotations

# definition of basic numeric type for convenience
NUMERIC = int | float

In [2]:
class Dual:
    def __init__(self, real: NUMERIC, dual: NUMERIC) -> None:
        self.real = real
        self.dual = dual

    @classmethod
    def coerce(cls, d: NUMERIC | "Dual") -> "Dual":
        # a real number is a dual number with zero dual part!
        return d if isinstance(d, Dual) else cls(d, 0)
    
    def __add__(self, other: NUMERIC | "Dual") -> "Dual":
        # (a + bε) + (c + dε) = (a + c) + (b + d)ε
        other = self.coerce(other)
        real = self.real + other.real
        dual = self.dual + other.dual
        return Dual(real, dual)

    def __radd__(self, other: NUMERIC | "Dual") -> "Dual":
        return self + other
    
    def __mul__(self, other: NUMERIC | "Dual") -> "Dual":
        # (a + bε) * (c + dε) = ac + (ad + bc)ε
        other = self.coerce(other)
        real = self.real * other.real
        dual = self.real * other.dual + self.dual * other.real
        return Dual(real, dual)

    def __rmul__(self, other: NUMERIC | "Dual") -> "Dual":
        return self * other

    def __sub__(self, other: NUMERIC | "Dual") -> "Dual":
        # subtraction is addition of the negative as per usual
        return self + (-1 * other)
    
    def __rsub__(self, other: NUMERIC | "Dual") -> "Dual":
        return other + (-1 * self)
    
    def __truediv__(self, other: NUMERIC | "Dual") -> "Dual":
        # since this is a bit more involved, we will set up the notation as in the note above
        other = self.coerce(other)
        a, b, c, d = self.real, self.dual, other.real, other.dual
        if c != 0:
            real = a / c
            dual = (b * c - a * d) / c / c
        else:
            if a == 0 and d != 0:
                real = b / d
                dual = 0
            else:
                raise ZeroDivisionError("Division by 'zero'!")
        return Dual(real, dual)
    
    def __rtruediv__(self, other: NUMERIC | "Dual") -> "Dual":
        other = self.coerce(other)
        return other / self
    
    def __pow__(self, other: NUMERIC | "Dual") -> "Dual":
        other = self.coerce(other)
        a, b, c, d = self.real, self.dual, other.real, other.dual
        # the only case where we'd allow a < 0 is if d = 0 and c is an integer
        if a < 0 and not (d == 0 and c.is_integer()):
            raise ValueError("Raising negative number to non-integer or dual power is undefined!")
        # we've handled the pathological case already, so safe to exponentiate and log here!
        if d == 0:
            real = a ** c
            dual = c * (a ** (c - 1)) * b
        else:
            real = a ** c
            dual = real * (b * c / a + d * np.log(a))
        return Dual(real, dual)
    
    def __rpow__(self, other: NUMERIC | "Dual") -> "Dual":
        other = self.coerce(other)
        return other ** self

    def __eq__(self, other: NUMERIC | "Dual") -> bool:
        # dual numbers are equal iff both their real and dual parts are equal
        other = self.coerce(other)
        return self.real == other.real and self.dual == other.dual
    
    def __repr__(self) -> str:
        return f"{self.real} + {self.dual}ε"


In [3]:
# now we can finally write our derivative function
# ideally this is done in a separate package, but we will keep it here under a namespace for simplicity

class FWAD:
    # we make the design choice to require the user to provide a function whose free parameters are resolved
    # prior to differentiation with respect to the target parameter
    # the alternative is handled by explicitly carrying the args and kwargs through
    @staticmethod
    def diff(func: Callable[[Dual], Dual]) -> Callable[[NUMERIC], NUMERIC]:
        def _diff(x: NUMERIC) -> NUMERIC:
            dual_x = Dual(x, 1)  # this is x + ε
            dual_result = func(dual_x)  # f(x + ε) = f(x) + f'(x)ε
            func_prime = dual_result.dual  # the dual part is the derivative at x!
            return func_prime
        return _diff
    
autodiff = FWAD()

# autodiff.diff(autodiff.diff(...(autodiff.diff(f))))(x) for nth order derivatives!

In [4]:
# to use autodiff, we need to define a soft wrapper over numpy functions so that they handle dual numbers
# ideally this is done in a separate package, but we will keep it here under a namespace for simplicity

class Dumpy:
    # we'll just do sine and exp for brevity. more can be added similarly
    @staticmethod
    def sin(x: NUMERIC | Dual) -> Dual:
        # sin(a + bε) = sin(a) + cos(a)bε
        x = Dual.coerce(x)
        real = np.sin(x.real)
        dual = np.cos(x.real) * x.dual
        return Dual(real, dual)

    @staticmethod
    def exp(x: NUMERIC | Dual) -> Dual:
        # exp(a + bε) = exp(a) + exp(a)bε
        x = Dual.coerce(x)
        real = np.exp(x.real)
        dual = real * x.dual
        return Dual(real, dual)

# alias for convenience and because it's cute :)
dp = Dumpy()

In [5]:
# examples! we'll set out the examples in the following cells and collect them using the validator at the end

# 1)
# let's do a nested horror: f(x) = exp(1/x) + exp(-2 * sin(x)) - x^5 + x^x
# this has: f'(x) = -exp(1/x)/x^2 - 2 * cos(x) * exp(-2 * sin(x)) - 5x^4 + x^x (1 + ln(x))

def f1(x: Dual) -> Dual:
    return dp.exp(1/x) + dp.exp(-2 * dp.sin(x)) - x ** 5 + x ** x


def f1_expected_derivative(x: NUMERIC) -> NUMERIC:
    return -np.exp(1/x) / x / x - 2 * np.cos(x) * np.exp(-2 * np.sin(x)) - 5 * x ** 4 + (x ** x) * (1 + np.log(x))


def f1_auto_derivative(x: NUMERIC) -> NUMERIC:
    return autodiff.diff(f1)(x)

print(f"expected: {f1_expected_derivative(1.2)}\n"
      f"actual:   {f1_auto_derivative(1.2)}")

expected: -10.606783408519464
actual:   -10.606783408519464


In [6]:
# 2)
# since we defined all arithmetic operations, this also works for functions defined with loops
# for simplicity, let's present f(x) = 2x^4 as a while loop
# we'll also look at its 2nd and 3rd derivatives, f''(x) = 24x^2 and f'''(x) = 48x

def f2(x: Dual) -> Dual:
    i = 0
    while i < 2:
        x *= x
        i += 1
    return 2 * x


def f2_expected_derivative(x: NUMERIC) -> NUMERIC:
    return 8 * x ** 3


def f2_expected_second_derivative(x: NUMERIC) -> NUMERIC:
    return 24 * x ** 2


def f2_expected_third_derivative(x: NUMERIC) -> NUMERIC:
    return 48 * x


def f2_auto_derivative(x: NUMERIC) -> NUMERIC:
    return autodiff.diff(f2)(x)


def f2_auto_second_derivative(x: NUMERIC) -> NUMERIC:
    return autodiff.diff(autodiff.diff(f2))(x)


def f2_auto_third_derivative(x: NUMERIC) -> NUMERIC:
    return autodiff.diff(autodiff.diff(autodiff.diff(f2)))(x)

print(f"expected (1): {f2_expected_derivative(0.23)}\n"
      f"actual (1):   {f2_auto_derivative(0.23)}\n"
      f"expected (2): {f2_expected_second_derivative(0.23)}\n"
      f"actual (2):   {f2_auto_second_derivative(0.23)}\n"
      f"expected (3): {f2_expected_third_derivative(0.23)}\n"
      f"actual (3):   {f2_auto_third_derivative(0.23)}")

expected (1): 0.097336
actual (1):   0.097336
expected (2): 1.2696
actual (2):   1.2696
expected (3): 11.040000000000001
actual (3):   11.040000000000001
